# Getting Started with this Repo

This notebook helps you get started with using this MF6RTM Aquifer Storage Recovery (ASR) Example repository.

This repo uses the conda for:
- Virtural environment managment
- Package managment for dependances
- Setting up the mf6rtm repo in "develop" mode.


# Installation and Setup

Create a custom conda virtual environment can be created using the `environment.yml` file included in this repo.

See the repo's [`README.md`](https://github.com/LimnoTech/mf6rtm-asr-example/blob/main/README.md#install-development-environment-with-conda) file for detailed instructions.

## Python Imports

In [1]:
import os
from pathlib import Path

import flopy
from modflowapi import ModflowApi
import phreeqcrm

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute


In [2]:
# Import the MFRTM package, installed using `conda develop`
import mf6rtm

display(mf6rtm.__file__)
try:
    # if current LimnoTech development version
    display(mf6rtm.__version__)
except AttributeError:
    pass

'/Users/aaufdenkampe/Documents/git_mf6rtm/mf6rtm/mf6rtm/__init__.py'

'0.1.11+externalio.dev'

NOTE: the notebook below runs from the `externalio` branch the upstream repo 
(Pablo's) https://github.com/p-ortega/mf6rtm/tree/feature/externalio

In [3]:
import utils # from this repo

### If you get `ModuleNotFoundError`

... you need to install this package into your environment using 
[`conda develop`](https://docs.conda.io/projects/conda-build/en/latest/resources/commands/conda-develop.html) 
command in your terminal with your local absolute path to the source directory
 of this repo. Then restart the kernel.

## Install `mf6rtm` using `conda develop`

In [4]:
# Find your current working directory, which should be folder for this notebook.
working_dir = Path.cwd()
# Find repository path (i.e. the parent to `/examples` directory for this notebook)
repo_path = working_dir
repo_path

PosixPath('/Users/aaufdenkampe/Documents/git_mf6rtm/mf6rtm-asr-example')

In [5]:
# If MF6RTM is installed in the same directory as this repository,
# then `mf6rtm_source_path` should be the source directory
mf6rtm_source_path = repo_path.parent / "mf6rtm"

print(
    "MF6RTM Source path exists?",
    mf6rtm_source_path.exists(),
    mf6rtm_source_path,
)

assert mf6rtm_source_path.exists(), "Find `mf6rtm` source path"

MF6RTM Source path exists? True /Users/aaufdenkampe/Documents/git_mf6rtm/mf6rtm


Use the Jupyter [`!` shell command](https://jakevdp.github.io/PythonDataScienceHandbook/01.05-ipython-and-shell-commands.html) to run the `conda develop {source_path}` terminal command directly from this notebook.

NOTE: The Jupyter [`%conda` magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-conda) will note work with `%conda develop {source_path}` in Windows, because it prepends the local working directory to the source path, inserting the wrong path to the `conda.pth` file.

In [6]:
!conda develop {mf6rtm_source_path}

path exists, skipping /Users/aaufdenkampe/Documents/git_mf6rtm/mf6rtm
completed operation for: /Users/aaufdenkampe/Documents/git_mf6rtm/mf6rtm


In [ ]:
# Also add this repository's source path for any local modules
!conda develop {repo_path / 'src'}

path exists, skipping /Users/aaufdenkampe/Documents/git_mf6rtm/mf6rtm-asr-example/src
completed operation for: /Users/aaufdenkampe/Documents/git_mf6rtm/mf6rtm-asr-example/src


If the path was added, Restart the kernel and rerun the cells above.

NOTE: if you have already done this once before, you may need to manually 
delete previous paths and add the path to:
`~/miniconda3/envs/modflow/lib/python3.12/site-packages/conda.pth`

## Alternative: add `mf6rtm` path directly to `conda.pth` file
Adapted from Clearwater Riverine examples\03_01_coupling_riverine_modules_nsm.ipynb

In [8]:
# Get path for active environment
active_env_path = Path(os.environ['CONDA_PREFIX'])
active_env_path 

PosixPath('/Users/aaufdenkampe/miniconda3/envs/modflow')

In [9]:
# Find site-packages folder in path for active environment
site_packages_folder = 'site-packages'
paths = []
for site_packages_path in active_env_path.rglob(site_packages_folder): # rglob for recursive search
    paths.append(site_packages_path)
paths

[PosixPath('/Users/aaufdenkampe/miniconda3/envs/modflow/lib/python3.12/site-packages')]

In [10]:
#create Path object for conda.pth file
conda_pth_filePath = site_packages_path / 'conda.pth'
 
#check if conda.pth file exists
if conda_pth_filePath.exists():
    print(f'The `conda.pth` file exists at {conda_pth_filePath}\n')
    print('It includes these contents')
    # Open the file in read mode ('r')
    with open(conda_pth_filePath, 'r') as file:
        # Read the entire content of the file
        file_contents = file.read()
        print(file_contents)
else:
    conda_pth_filePath.parent.mkdir(parents=True, exist_ok=True)
    with open(conda_pth_filePath, 'a'):
        print('conda.pth file created')

The `conda.pth` file exists at /Users/aaufdenkampe/miniconda3/envs/modflow/lib/python3.12/site-packages/conda.pth

It includes these contents
/Users/aaufdenkampe/Documents/git_mf6rtm/mf6rtm
/Users/aaufdenkampe/Documents/git_mf6rtm/mf6rtm-asr-example/src



In [11]:
# add needed path info to conda.pth file, if necessary
with open(conda_pth_filePath, 'a+') as file:
    # --- Read existing content ---
    # To read, you must first move the cursor to the beginning of the file
    file.seek(0)
    file_contents = file.read()
    if str(mf6rtm_source_path) in file_contents:    
        print(f'conda.pth file already includes {mf6rtm_source_path}')
    else:
    # --- Append new content ---
        file.seek(0)
        file.write(str(mf6rtm_source_path))
        file.write('\n')
        print(f'conda.pth file has been modified by adding {mf6rtm_source_path}')

conda.pth file already includes /Users/aaufdenkampe/Documents/git_mf6rtm/mf6rtm


# Confirm Paths to MF6 Executable & Library
Different versions can be downloaded from: https://github.com/MODFLOW-ORG/executables to a folder similar to this: `bin/mf6.5.0/macarm` 

On Mac, will need to give permissions with these terminal commands from the 
```sh
xattr -dr com.apple.quarantine mf6
xattr -dr com.apple.quarantine libmf6.dylib
```


In [12]:
# Check Modflow 6 version installed with ModflowAPI
try:
    mf6_exe = "mf6"
    dll = "libmf6"
    mf6_version = !{mf6_exe} --version
    mf6dll_version = ModflowApi("libmf6").get_version()
    print(f"Executable & library installed with modflowapi: {mf6_version[1]}, dll: {mf6dll_version}")
except Exception:
    print("Modflow executables not found in environment")

Executable & library installed with modflowapi: mf6: 6.6.3 09/29/2025, dll: 6.6.3


In [13]:
# Select option to use alternate versions
use_version_installed_with_modflowapi = False

# user = "Laren"
user = "Anthony"

# version = "6.4.2"
version = "6.5.0"

os = "macarm"


if use_version_installed_with_modflowapi:
    print(f"Using executable installed with modflowapi: {mf6_version[1]}")
else:
    if user == "Lauren":
        # If using executable from GMS
        mf6_bin_path = Path(r"C:/program files/gms 10.8 64-bit/python/lib/site-packages/xms/executables/modflow6")
        mf6_exe = mf6_bin_path / "mf6.exe"
        dll = mf6_bin_path / "libmf6.dll"
    elif user == "Anthony":
        mf6_bin_path = repo_path / "bin" / f"mf{version}" / os
        mf6_exe = mf6_bin_path / "mf6"
        dll = mf6_bin_path / "libmf6.dylib"
    else:
        print("Create a new user and set paths to mf6 and libmf6")

mf6_version = !{mf6_exe} --version
mf6dll_version = ModflowApi(dll).get_version()
print(f"User-selected executable ({mf6_exe.exists()}): {mf6_version[1]}, dll: {mf6dll_version}")

User-selected executable (True): mf6: 6.5.0 05/23/2024, dll: 6.5.0


# END
You are ready to use the other notebooks in this repo.